In [112]:
#-*- coding:utf-8 -*-
import pandas as pd
import numpy as np
import pymysql
import time
from string import Template

In [113]:
# basic filter data  
conn = pymysql.connect(host='119.3.6.221', port=3306, user='root',
                       passwd='citydo@123', db='aqhy', charset="utf8")
cursor = conn.cursor()

sqlFilter = '''SELECT resolution,cpu_type,cpu_name,cpu_number,access_app,os_isIOS from aqhy.s_device_model
               group by resolution,cpu_type,cpu_name,cpu_number,access_app,os_isIOS;
            '''
filt = pd.read_sql_query(sqlFilter, conn)

In [114]:
filt.head(2)

,resolution,cpu_type,cpu_name,cpu_number,access_app,os_isIOS
0,逻辑：667 × 375; 物理：2001 × 1125,CPU_TYPE_ARM32,Apple A100,2,FaceCheck1,1
1,逻辑：667 × 375; 物理：2001 × 1125,CPU_TYPE_ARM32,Apple A110,2,FaceCheck1,1


In [115]:
def getSql(filter):
    # 通过传入的filt参数获取相应的数据
    sql = Template(
            '''select * from aqhy.s_device_model where resolution="$resolution" and cpu_type="$cputype"  and cpu_name="$cpuname"
               and cpu_number=$cpunumber and access_app="$accessapp" and os_isIOS=$os;
            ''').substitute(
         {"resolution": filter[0],"cputype": filter[1], "cpuname": filter[2], 
          "cpunumber": filter[3],"accessapp": filter[4],"os": filter[5]})
    return sql

In [116]:
sql = getSql(filt.iloc[1,:])
rsl = pd.read_sql_query(sql, conn)
rsl.head(2)

,id,uuid,ram,is_emulator,is_jailbreak,is_proxy,is_root,is_vm,is_vpn,os_version,...,net_isOther,os_isAndroid,os_isIOS,carrier_info_isChinaUnion,carrier_info_isChinaTelecom,carrier_info_isChinaMobile,carrier_info_isUnknown,carrier_info_isOther,create_time,update_time
0,4001,1c903008721a4091b185da947b120bc4,0.9453266,0,1,1,0,1,1,9,...,0,0,1,0,0,1,0,0,2018-12-12 17:45:47,2018-12-12 17:45:47
1,4002,1c903008721a4091b185da947b120bc4,0.9453266,0,0,0,0,0,0,9,...,0,0,1,0,0,1,0,0,2018-12-12 17:45:47,2018-12-12 17:45:47


In [117]:
tunk = []
for k in range(filt.shape[0]):
    strings = ''.join(str(i) for i in filt.iloc[k,:])
    tunk.append(strings)
dictModel = {val:i for i,val in enumerate(tunk)}
# dictModel

In [119]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pickle
import json

# 生成模型  'is_vpn', 'os_version' 此处删除了os_version
select = ['uuid', 'ram', 'is_emulator', 'is_jailbreak', 'is_proxy','is_root', 'is_vm', 'is_vpn',  
          'access_time_zone','access_time_value', 'brand_hasChn', 'brand_length','brand_hasSpechars', 'disk_space',
          'ip_p1', 'ip_p2', 'ip_p3', 'ip_p4','platform_Eng_num', 'platform_Num_num', 'platform_hasSpechars','lat_integer', 
          'lat_decimal', 'lng_integer', 'lng_decimal', 'net_is2G','net_is3G', 'net_is4G', 'net_isWifi', 'net_isOther',
          'carrier_info_isChinaUnion', 'carrier_info_isChinaTelecom','carrier_info_isChinaMobile', 'carrier_info_isUnknown',
          'carrier_info_isOther']

for i in range(filt.shape[0]):
    # 生成sql，获取数据
    sql = getSql(filt.iloc[i,:])
    train = pd.read_sql_query(sql, conn)
    train = train[select]
    #dictId = 'DI' + str(i)
    ID = {val:i for i,val in enumerate(set(train['uuid']))}
    DI = {i:val for i,val in enumerate(set(train['uuid']))}
    train['uuid'] = train['uuid'].map(ID)
    x_train,x_test,y_train,y_test = train_test_split(train.iloc[:,1:],train.iloc[:,0],test_size=0.3,random_state=i,stratify=train.iloc[:,0])
    model = RandomForestClassifier(n_estimators=10, max_depth=None,min_samples_split=2, random_state=i)
    model = model.fit(x_train,y_train)
    print("Training score:%f"%(model.score(x_train,y_train)))
    print("Testing score:%f"%(model.score(x_test,y_test)))
    # save model
    mapid = r'C:\Users\fab\Desktop\jupyter\userid\test\model\\'+ 'ID' + str(i) + '.json'
    name = r'C:\Users\fab\Desktop\jupyter\userid\test\model\\'+ 'model' + str(i) + '.pickle'
    # save dict
    with open(mapid,'w') as f:
        json.dump(DI,f)
    #save model
    with open(name, 'wb') as fw:
        pickle.dump(model,fw)
    # load model
    with open(name, 'rb') as fr:
        newModel = pickle.load(fr)
    print("load model score:%f"%(newModel.score(x_test,y_test)))

Training score:1.000000
Testing score:1.000000
load model score:1.000000
Training score:1.000000
Testing score:1.000000
load model score:1.000000
Training score:1.000000
Testing score:1.000000
load model score:1.000000
Training score:1.000000
Testing score:1.000000
load model score:1.000000
Training score:1.000000
Testing score:1.000000
load model score:1.000000
Training score:1.000000
Testing score:1.000000
load model score:1.000000
Training score:1.000000
Testing score:1.000000
load model score:1.000000
Training score:1.000000
Testing score:1.000000
load model score:1.000000
Training score:1.000000
Testing score:1.000000
load model score:1.000000


In [136]:
# test
msg = {'id':4002, 'uuid':'1c903008721a4091b185da947b120bc4', 'ram':0.9453266, 'is_emulator':0, 'is_jailbreak':0, 'is_proxy':0,
       'is_root':0, 'is_vm':0, 'is_vpn':0, 'os_version':'9', 'resolution':'逻辑：667 × 375; 物理：2001 × 1125', 'cpu_name':'Apple A110',
       'cpu_number':'2', 'cpu_type':'CPU_TYPE_ARM32', 'access_app':'FaceCheck1', 'access_time_zone':0,'access_time_value':0.0025, 
       'brand_hasChn':1, 'brand_length':0.2,'brand_hasSpechars':0, 'disk_space':1.21302, 'ip_p1':0.003906, 'ip_p2':0.003906, 
       'ip_p3':0.003906, 'ip_p4':0.003906,'platform_Eng_num':0.5, 'platform_Num_num':0.1, 'platform_hasSpechars':1,
       'lat_integer':0.39, 'lat_decimal':0.53, 'lng_integer':0.7, 'lng_decimal':0.57,'wifi_Chn_num':0, 'wifi_Num_Char_num':1,
       'wifi_Spechars_num':0, 'net_is2G':0,'net_is3G':1, 'net_is4G':0, 'net_isWifi':0, 'net_isOther':0, 'os_isAndroid':0,
       'os_isIOS':1, 'carrier_info_isChinaUnion':0, 'carrier_info_isChinaTelecom':0,'carrier_info_isChinaMobile':1, 
       'carrier_info_isUnknown':0,'carrier_info_isOther':0}
tag = msg['resolution'] + msg['cpu_type'] + msg['cpu_name'] + str(msg['cpu_number']) + msg['access_app'] + str(msg['os_isIOS'])
#dictModel[tag]
select = ['ram', 'is_emulator', 'is_jailbreak', 'is_proxy','is_root', 'is_vm', 'is_vpn',  
          'access_time_zone','access_time_value', 'brand_hasChn', 'brand_length','brand_hasSpechars', 'disk_space',
          'ip_p1', 'ip_p2', 'ip_p3', 'ip_p4','platform_Eng_num', 'platform_Num_num', 'platform_hasSpechars','lat_integer', 
          'lat_decimal', 'lng_integer', 'lng_decimal', 'net_is2G','net_is3G', 'net_is4G', 'net_isWifi', 'net_isOther',
          'carrier_info_isChinaUnion', 'carrier_info_isChinaTelecom','carrier_info_isChinaMobile', 'carrier_info_isUnknown',
          'carrier_info_isOther']
req = np.array([msg[sl] for sl in select]).reshape(1,-1)
path = r'C:\Users\fab\Desktop\jupyter\userid\test\model\\'+ 'model' + str(dictModel[tag]) + '.pickle'
DIpath = r'C:\Users\fab\Desktop\jupyter\userid\test\model\\'+ 'ID' + str(dictModel[tag]) + '.json'
print(DIpath)
with open(path, 'rb') as fr:Model = pickle.load(fr)
fdi = open(DIpath, encoding='utf-8')
DI = json.load(fdi)
print("predict uuid:",DI[str(Model.predict(req)[0])])
print("max proba:",max(Model.predict_proba(req)))

C:\Users\fab\Desktop\jupyter\userid\test\model\\ID1.json
predict uuid: 1c903008721a4091b185da947b120bc4
max proba: [1. 0.]
